### Exploratory Data Analysis
ensure `SA2_2021_AUST_SHP_GDA94` and `requirements.txt` are to the same directory as this Jupyter notebook file 

for linux command, run this beforehand:
```
sudo apt-get install gdal-bin
sudo apt-get install libgdal-dev

sudo apt install python-geopandas
```

In [1]:
!pip install -r ./requirements.txt

In [2]:
import pandas as pd
import geopandas as gpd
import json
import pyproj
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash

from dash import Dash, html, dcc
from dash.dependencies import Input, Output

import couchdb

from datetime import datetime

MAPBOX_ACCESS_TOKEN="pk.eyJ1Ijoia3VrYWhlYWRsYSIsImEiOiJjbDJ5Mml5MHEweHlkM2tvNWxodm1najcwIn0.h1p7x_rboO1iqycGcogJFQ"

### Exploratory Data Analysis for `Historical Tweets`

In [3]:
USER = 'user'
PASSWORD = 'password'

server = couchdb.Server('http://{}:{}@172.26.134.34:5984/'.format(USER, PASSWORD))

In [4]:
db = server['new_tweets']
envir_df = server['envir_test1']

In [5]:
# envir_db.view('area_week/area_week_topic')
topic_agg = db.view('area_week/area_week_topic', include_doc=True)
topic_df = pd.DataFrame((row.key+[row.value['sentiments']['compound']] for row in topic_agg),
                        columns = ['time', 'area', 'topic','sentiment'])

# Aggregate by average sentiment 
topic_df = topic_df.groupby(['time', 'area', 'topic']).mean().reset_index()

#filter out wNaN 
topic_df['week'] = topic_df.apply(lambda x: x['time'].split('-')[0], axis=1)
topic_df = topic_df[topic_df['week'] != 'wNaN']

#Change string to datetime format and filtering out non-valid string
topic_df['time'] = topic_df.apply(lambda row: datetime.strptime(row['time']+'-1', 'w%W-%Y-%w'), axis=1) #append -1 as Monday
topic_df = topic_df[topic_df['area'] != 'zzzzzzzzz'] 

# print("topic_df Shape:", topic_df.shape)
# topic_df.head()

topic_df Shape: (475, 5)


,time,area,topic,sentiment,week
0,2018-01-01,lyneham,health,0.9667,w1
2,2020-01-06,berry - kangaroo valley,environment,0.3818,w1
3,2021-01-04,guildford west - merrylands west,environment,-0.0108,w1
4,2021-01-04,guildford west - merrylands west,health,0.6124,w1
5,2021-01-04,springvale south,environment,0.0000,w1


In [6]:
sentiment_agg = db.view('sentiments/area_week_avg_compound', group_level=2, include_doc=True)
sentiment_df = pd.DataFrame([row.key+row.value for row in sentiment_agg], 
                            columns=['time', 'area', 'sentiment', 'count'])

# Remove NaN value
sentiment_df['week'] = sentiment_df.apply(lambda x: x['time'].split('-')[0], axis=1)
sentiment_df = sentiment_df[sentiment_df['week'] != 'wNaN']

# Change string to datetime format
sentiment_df['time'] = sentiment_df.apply(lambda row: datetime.strptime(row['time']+'-1', 'w%W-%Y-%w'), axis=1)

# # Filter out only Melbourne and Sydney
# sentiment_df = sentiment_df[(sentiment_df['area']== 'melbourne') | (sentiment_df['area'] == 'sydney')]

#remove null_areas
sentiment_df = sentiment_df[sentiment_df['area'] != 'zzzzzzzzz'] 

# print("topic_df Shape:", sentiment_df.shape)
# sentiment_df.head()

topic_df Shape: (417, 5)


,time,area,sentiment,count,week
0,2018-01-01,lyneham,0.96670,1,w1
2,2020-01-06,berry - kangaroo valley,0.38180,1,w1
3,2021-01-04,guildford west - merrylands west,0.30080,2,w1
4,2021-01-04,springvale south,0.15262,5,w1
5,2021-01-04,sydney (north) - millers point,0.78390,1,w1


In [7]:
sa2_gdf = gpd.read_file("./SA2_2021_AUST_SHP_GDA94")
sa2_gdf['SA2_NAME21'] = sa2_gdf['SA2_NAME21'].str.lower()
sa2_gdf.to_crs(pyproj.CRS.from_epsg(4283), inplace=True)
# sa2_gdf.head()

,SA2_CODE21,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry
0,101021007,braidwood,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,3418.3525,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.58423 -35.44427, 149.58444 -35.4..."
1,101021008,karabar,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,6.9825,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.21898 -35.36739, 149.21799 -35.3..."
2,101021009,queanbeyan,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,4.7620,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.21326 -35.34326, 149.21619 -35.3..."
3,101021010,queanbeyan - east,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,13.0032,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.24033 -35.34782, 149.24023 -35.3..."
4,101021012,queanbeyan west - jerrabomberra,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,13.6748,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.19572 -35.36128, 149.19970 -35.3..."


In [8]:
sa2_gdf_sent = sa2_gdf.copy()
sentiment_geo_df = sentiment_df.merge(sa2_gdf_sent, left_on='area', right_on='SA2_NAME21', how='left')
sentiment_gdf = gpd.GeoDataFrame(sentiment_geo_df, crs="EPSG:4283", geometry=sentiment_geo_df.geometry)
# sentiment_gdf.head()

,time,area,sentiment,count,week,SA2_CODE21,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,...,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry
0,2018-01-01,lyneham,0.96670,1,w1,801051057,lyneham,0,No change,80105,...,Australian Capital Territory,8ACTE,Australian Capital Territory,8,Australian Capital Territory,AUS,Australia,5.4776,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.11775 -35.23629, 149.11816 -35.2..."
1,2020-01-06,berry - kangaroo valley,0.38180,1,w1,114011272,berry - kangaroo valley,0,No change,11401,...,Southern Highlands and Shoalhaven,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,537.0884,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((150.41186 -34.75384, 150.41140 -34.7..."
2,2021-01-04,guildford west - merrylands west,0.30080,2,w1,125031484,guildford west - merrylands west,0,No change,12503,...,Sydney - Parramatta,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,5.4654,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((150.96026 -33.84519, 150.96039 -33.8..."
3,2021-01-04,springvale south,0.15262,5,w1,212041318,springvale south,0,No change,21204,...,Melbourne - South East,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,4.5600,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((145.13575 -37.96907, 145.13531 -37.9..."
4,2021-01-04,sydney (north) - millers point,0.78390,1,w1,117031644,sydney (north) - millers point,1,New,11703,...,Sydney - City and Inner South,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,3.2122,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"MULTIPOLYGON (((151.22538 -33.85526, 151.22524..."


In [9]:
sentiment_overall_agg = sentiment_gdf.groupby(['GCC_NAME21']).sum().reset_index()
# sentiment_overall_agg.head()

,GCC_NAME21,sentiment,count,AREASQKM21
0,Australian Capital Territory,11.673042,37,1442.4061
1,Greater Adelaide,1.042166,117,68.7461
2,Greater Brisbane,2.136417,28,19.9746
3,Greater Darwin,0.726900,1,30.2554
4,Greater Hobart,0.583746,27,266.2794


In [10]:
sentiment_overall_avg = sentiment_gdf.groupby([sentiment_gdf.time.dt.year, sentiment_gdf.GCC_NAME21]).mean().reset_index()
# sentiment_overall_avg.head()

,time,GCC_NAME21,sentiment,count,AREASQKM21
0,2016,Greater Brisbane,0.299367,3.0,3.3873
1,2016,Greater Hobart,0.467150,2.0,88.7598
2,2016,Greater Melbourne,0.125233,1.0,4.5600
3,2016,Rest of Qld,0.256433,3.0,6.2058
4,2017,Greater Adelaide,0.571900,2.0,7.5686


In [11]:
sentiment_overall_avg = sentiment_df.groupby([sentiment_gdf.time.dt.year, sentiment_gdf.GCC_NAME21]).mean().reset_index()
melb_sentiment_now = sentiment_overall_avg.loc[
                        (sentiment_overall_avg['GCC_NAME21'] == 'Greater Melbourne') & (sentiment_overall_avg['time'] == datetime.now().year),
                        'sentiment'].item()
syd_sentiment_now = sentiment_overall_avg.loc[
                        (sentiment_overall_avg['GCC_NAME21'] == 'Greater Sydney') & (sentiment_overall_avg['time'] == datetime.now().year),
                        'sentiment'].item()
melb_sentiment_prev = sentiment_overall_avg.loc[
                        (sentiment_overall_avg['GCC_NAME21'] == 'Greater Melbourne') & (sentiment_overall_avg['time'] == datetime.now().year-1),
                        'sentiment'].item()
syd_sentiment_prev = sentiment_overall_avg.loc[
                        (sentiment_overall_avg['GCC_NAME21'] == 'Greater Sydney') & (sentiment_overall_avg['time'] == datetime.now().year-1),
                        'sentiment'].item()
#Overall Sentiment
melb_sentiment_overall = sentiment_overall_avg.loc[
                        (sentiment_overall_avg['GCC_NAME21'] == 'Greater Melbourne'),
                        'sentiment'].mean()
syd_sentiment_overall = sentiment_overall_avg.loc[
                        (sentiment_overall_avg['GCC_NAME21'] == 'Greater Sydney'),
                        'sentiment'].mean()

In [12]:
# topic_df.topic.unique()

In [13]:
pie_fig = px.pie(sentiment_overall_agg, values='count', 
            names='GCC_NAME21', 
            title='Sentiment Count by Area',
            color_discrete_sequence=px.colors.sequential.Peach,
            height=616,
            hole=0.4)
pie_fig.update_layout(
                margin={"r":0,"t":50,"l":0,"b":0},
                paper_bgcolor='rgba(0,0,0,0)', 
                plot_bgcolor='rgba(0,0,0,0)',
                font_color='#FFFFFF')
# pie_fig.show() #comment this out for flask app

In [14]:
indicator_fig = go.Figure()
indicator_fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = melb_sentiment_now,
    title = {"text": "Greater Melbourne<br> Avg. Sentiment Now<br><span style='font-size:0.8em;color:gray'>Compare to previous year</span>"},
    delta = {'reference': melb_sentiment_prev},
    domain = {'row': 0, 'column': 0}))
indicator_fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = syd_sentiment_now,
    delta = {'reference': syd_sentiment_prev},
    title = {"text": "Greater Sydney<br> Avg. Sentiment Now<br><span style='font-size:0.8em;color:gray'>Compare to previous year</span>"},
    domain = {'row': 0, 'column': 1}))
indicator_fig.add_trace(go.Indicator(
    mode = "gauge+number",
    value = melb_sentiment_overall,
    gauge = {
        'axis': {'range': [min(sentiment_df['sentiment']), max(sentiment_df['sentiment'])]}, 
        'bar': {'color': "orange"}},
    title = {"text": "Greater Melboourne <br>Overall Sentiment"},
    domain = {'row': 0, 'column': 2}))
indicator_fig.add_trace(go.Indicator(
    mode = "gauge+number",
    value = syd_sentiment_overall,
    gauge = {
        'axis': {'range': [min(sentiment_df['sentiment']), max(sentiment_df['sentiment'])]}, 
        'bar': {'color': "orange"}},
    title = {"text": "Greater Sydney <br>Overall Sentiment"},
    domain = {'row': 0, 'column': 3}))

indicator_fig.update_layout(
    grid = {'rows': 1, 'columns': 4, 'pattern': "independent"},
    paper_bgcolor='rgba(0,0,0,0)', 
    plot_bgcolor='rgba(0,0,0,0)',
    font_color='#FFFFFF')
# indicator_fig.show() # comment this line for flask app

In [15]:
sa2_gdf_topic = sa2_gdf.copy()
topic_df = topic_df.merge(sa2_gdf_topic, left_on='area', right_on='SA2_NAME21', how='left')
topic_df = gpd.GeoDataFrame(topic_df, crs="EPSG:4283", geometry=topic_df.geometry)
# topic_df.head() # comment this out for flask app

,time,area,topic,sentiment,week,SA2_CODE21,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,...,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry
0,2018-01-01,lyneham,health,0.9667,w1,801051057,lyneham,0,No change,80105,...,Australian Capital Territory,8ACTE,Australian Capital Territory,8,Australian Capital Territory,AUS,Australia,5.4776,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.11775 -35.23629, 149.11816 -35.2..."
1,2020-01-06,berry - kangaroo valley,environment,0.3818,w1,114011272,berry - kangaroo valley,0,No change,11401,...,Southern Highlands and Shoalhaven,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,537.0884,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((150.41186 -34.75384, 150.41140 -34.7..."
2,2021-01-04,guildford west - merrylands west,environment,-0.0108,w1,125031484,guildford west - merrylands west,0,No change,12503,...,Sydney - Parramatta,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,5.4654,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((150.96026 -33.84519, 150.96039 -33.8..."
3,2021-01-04,guildford west - merrylands west,health,0.6124,w1,125031484,guildford west - merrylands west,0,No change,12503,...,Sydney - Parramatta,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,5.4654,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((150.96026 -33.84519, 150.96039 -33.8..."
4,2021-01-04,springvale south,environment,0.0000,w1,212041318,springvale south,0,No change,21204,...,Melbourne - South East,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,4.5600,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((145.13575 -37.96907, 145.13531 -37.9..."


In [16]:
topic_df_overtime = topic_df.groupby([topic_df.time.dt.year, topic_df.topic]).mean().reset_index()
# topic_df_overtime[topic_df_overtime.topic == "environment"] 

In [17]:
lin_fig = go.Figure()

for topic in topic_df_overtime.topic.unique():
    lin_fig.add_trace(go.Scatter(
        x=topic_df_overtime[topic_df_overtime.topic == topic]['time'],
        y=topic_df_overtime[topic_df_overtime.topic == topic]['sentiment'],
        name=topic,
        connectgaps=True
    ))

lin_fig.update_layout(
    title="Sentiment by Topic Overtime",
    xaxis_title="Year",
    yaxis_title="Sentiment",
    paper_bgcolor='rgba(0,0,0,0)', 
    plot_bgcolor='rgba(0,0,0,0)',
    font_color='#FFFFFF'
)
# lin_fig.show() #comment this for flask app

Considering  `envir_test1` data wrangglings below (feel fee to ignore)

In [18]:
# envir_topic = envir_db.view('area_week/area_week_topic', include_doc=True)
# envir_topic_df = pd.DataFrame((row.key+[row.value['sentiments']['compound']] for row in envir_topic),
#                         columns = ['time', 'area', 'topic','sentiment'])
# envir_topic_df = envir_topic_df.groupby(['time', 'area', 'topic']).mean().reset_index()
# envir_topic_df['time'] = envir_topic_df.apply(lambda row: datetime.strptime(row['time']+'-1', 'w%W-%Y-%w'), axis=1) #append -1 as Monday
# envir_topic_df = envir_topic_df[envir_topic_df['area'] != 'zzzzzzzzz']
# envir_topic_df.head()

In [19]:
# # envir_topic_df = envir_topic_df.merge(sa2_gdf, left_on='area', right_on='SA2_NAME21', how='left')
# envir_topic_overall_df = envir_topic_df.groupby(['area', 'topic']).mean().reset_index()
# # envir_topic_overall_df.head()

# envir_topic_overall_df = envir_topic_overall_df.merge(sa2_gdf, left_on='area', right_on='SA2_NAME21', how='left')
# envir_topic_overall_df

In [20]:
# # envir_topic_overall.to_crs(pyproj.CRS.from_epsg(4283), inplace=True)
# envir_topic_overall_df = gpd.GeoDataFrame(envir_topic_overall_df, crs="EPSG:4283", geometry=envir_topic_overall_df.geometry)
# # envir_topic_overall_df.set_index('area', inplace=True)
# envir_topic_overall_df.head()

In [21]:
# fig = px.choropleth_mapbox(envir_topic_overall_df, geojson=envir_topic_overall_df.geometry, 
#                             locations=envir_topic_overall_df.index,
#                             color=envir_topic_overall_df.sentiment,
#                             zoom= 5,
#                             center = {"lat": -37.8136, "lon": 144.9631}, mapbox_style="carto-positron")
# fig.update_layout(mapbox_style="dark", mapbox_accesstoken=MAPBOX_ACCESS_TOKEN)
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()

### Using `plotly.graph_objects` as alternative, uncomment those for substitutes

In [22]:
# envir_topic_df = envir_topic_df.merge(sa2_gdf, left_on='area', right_on='SA2_NAME21', how='left')
# envir_topic_df = gpd.GeoDataFrame(envir_topic_df, crs="EPSG:4283", geometry=envir_topic_df.geometry)
# envir_topic_df.crs

In [23]:
# dff = topic_df.copy()
# gdff = dff[dff['time'].dt.year == 2022]
# gdff = gdff[gdff['topic'] == 'health']
# gdff.set_index('area', inplace=True)
# gdff.shape

In [24]:
# fig = px.choropleth_mapbox(gdff, 
#                         geojson=gdff.geometry,
#                         color=gdff.sentiment,
#                         locations=gdff.index,
#                         zoom=5,
#                         center = {"lat": -37.8136, "lon": 144.9631},
#                         mapbox_style="carto-positron")
# fig.show()

In [25]:
# geometry_json = json.loads(gdff.geometry.to_json())
# fig = go.Figure(
#     data=go.Choroplethmapbox(geojson=geometry_json,
#                             locations=gdff.index,
#                             z=gdff.sentiment,
#                             zmin=min(gdff.sentiment), zmax=max(gdff.sentiment),
#                             colorscale="Oranges",
#                             marker_opacity=0.5, marker_line_width=0)
# )
# fig.update_layout(mapbox_style="dark", 
#                 mapbox_accesstoken=MAPBOX_ACCESS_TOKEN,
#                 mapbox_center = {"lat": -37.8136, "lon": 144.9631},
#                 mapbox_zoom=3)
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, 
#                 paper_bgcolor='rgba(0,0,0,0)', 
#                 plot_bgcolor='rgba(0,0,0,0)',
#                 font_color='#FFFFFF')
# fig.show()

In [30]:
drop_down_lst = list(map(lambda x: x.capitalize(), topic_df.topic.unique()))
left_container = html.Div(
                id="left-container",
                children=[
                    dcc.Dropdown(drop_down_lst , drop_down_lst[0],
                                id='topic-dropdown',
                                style={'font-family':'sans-serif', 
                                    'margin': '0px 0px 5px 0px', 
                                    'padding': '5px 0px 5px 0px',
                                    'background-color':'#282828'},
                                clearable=False
                    ),
                    dcc.Graph(id='choropleth-mapbox', figure={}),
                    html.Div(
                        children=[
                            html.P(
                                id="slider-text",
                                children="Drag the slider to change the year:",
                                style={'color':'#FFECE8', 'font-family':'sans-serif'}
                            ),
                            dcc.Slider(
                                    id='year-slider',
                                    min=min(topic_df.time).year,
                                    max=max(topic_df.time).year,
                                    value=max(topic_df.time).year,
                                    marks={str(year): {
                                        'label':str(year), 
                                        'style': {"color": "#FDA172", 'font-family':'sans-serif'} 
                                    } for year in range(min(topic_df.time).year, max(topic_df.time).year+1, 1)},
                                    step=None
                            )
                        ],
                        style={'padding': '20px 20px 20px 20px', 'background-color':'#282828'}),
                    html.Div(id='output-container', children=[], 
                            style={'text-align': 'center', 
                                'color':'#FDA172', 
                                'padding':'10px',
                                'display': 'none'
                            },)
                ], style={'display': 'inline-block', 'width':'49%', 'vertical-align':'top'})

In [31]:
right_container = html.Div([
    dcc.Graph(id='tweet-distb-by-city', figure=pie_fig),
], style={'display': 'inline-block', 
        'width': '49%', 
        'margin': '0px 0px 0px 10px', 
        'padding': '10px 10px 20px 18.5px',
        'verticle-align':'top', 
        'text-align': 'center', 
        'background-color':'#282828'})

In [32]:
# For Jupyter notebook uncomment the following line
app = JupyterDash(__name__)

# For flask app uncomment the following line
# app = Dash(__name__)

app.layout = html.Div([
    html.H1('Sentiment Transportation on Dashboard', 
                            style={'text_align': 'center', 
                                'color':'#FDA172', 
                                'font-family':'sans-serif',
                                'margin': '1em 0.3em 0.8em 0.3em'}),
    html.Div(id="heading-description", children=[
        html.H2('This dashboard shows the sentiment of selected topic in Australia. (+ve means happy, -ve mean sad).', 
            style={'text_align': 'center', 'color':'#FFFFFF', 'font-family':'sans-serif', 'margin': '0.7em'}),
        html.P('The data were product from MapReduce for Tweets that have location enabled and text which are able to be parsed into a Machine Learning Sentiment Scorer',
            style={'text_align': 'center', 'color':'#FFFFFF', 'font-family':'sans-serif', 'margin': '0.8em 0.8em 1.2em 0.8em'})
    ], style={'border-left': '5px solid darkorange'}),
    dcc.Graph(id='indicators', 
        figure=indicator_fig, 
        style={
            'width':'100%',
            'background-color':'#282828',
            'margin': '8px 0px 8px 0px',
        }),
    html.Div(children=[
        left_container,
        right_container
    ], style={'width': '100%', 'margin-bottom': '10px'}),
    dcc.Graph(id='tweet-sentiment-overtime', figure=lin_fig, style={'background-color':'#282828'}),
])

@app.callback(
    [
     Output(component_id='output-container', component_property='children'),
     Output(component_id='choropleth-mapbox', component_property='figure')
    ],
    [Input(component_id='year-slider', component_property='value'),
     Input(component_id='topic-dropdown', component_property='value')]
)
def update_graph(year, topic):
    container = "The year selected was: {}".format(year)
    
    dff = topic_df.copy()
    gdff = dff[dff['time'].dt.year == year]
    gdff = gdff[gdff['topic'] == topic.lower()]
    gdff.set_index('area', inplace=True)
    gdff.index = gdff.index.str.capitalize()

    ##### -- Uncomment below for plotly express, altough it renders slow
    # fig = px.choropleth_mapbox(gdff, 
    #                     geojson=gdff.geometry,
    #                     color=gdff.sentiment,
    #                     locations=gdff.index,
    #                     zoom=5,
    #                     center = {"lat": -37.8136, "lon": 144.9631},
    #                     mapbox_style="carto-positron",
    #                     color_continuous_scale="oranges")
    # fig.update_layout(mapbox_style="dark", mapbox_accesstoken=MAPBOX_ACCESS_TOKEN)
    # fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, 
    #                 paper_bgcolor='rgba(0,0,0,0)', 
    #                 plot_bgcolor='rgba(0,0,0,0)',
    #                 font_color='#FFFFFF')
    #####-- End of plotly express

    geometry_json = json.loads(gdff.geometry.to_json())
    fig = go.Figure(
        data=go.Choroplethmapbox(geojson=geometry_json,
                                locations=gdff.index,
                                z=gdff.sentiment,
                                zmin=min(gdff.sentiment), zmax=max(gdff.sentiment),
                                colorscale="Oranges",
                                marker_opacity=0.5, marker_line_width=0)
    )
    fig.update_layout(mapbox_style="dark", 
                    mapbox_accesstoken=MAPBOX_ACCESS_TOKEN,
                    mapbox_center = {"lat": -37.8136, "lon": 144.9631},
                    mapbox_zoom=3)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, 
                    paper_bgcolor='rgba(0,0,0,0)', 
                    plot_bgcolor='rgba(0,0,0,0)',
                    font_color='#FFFFFF')
    return container, fig
    # return fig
    
app.run_server(mode="external", debug=True)

Dash app running on http://127.0.0.1:8050/
